# Chapter 3 (Acrobot)
### Author: Luis Mantilla (@BestQuark)
This code was written for my undergraduate mathematics thesis on Deep Q-learning under the supervision of Dr. Mauricio Junca.

In [ ]:
#Reinforcement learning dependences
!pip install keras-rl2 tensorflow gym > /dev/null 2>&1
!apt-get install -y libglu1-mesa-dev libgl1-mesa-dev libosmesa6-dev xvfb ffmpeg curl patchelf libglfw3 libglfw3-dev cmake zlib1g zlib1g-dev swig > /dev/null 2>&1
!pip install 'gym[all]' > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
! wget http://www.atarimania.com/roms/Roms.rar > /dev/null 2>&1
! mkdir /content/ROM/ > /dev/null 2>&1
! unrar e /content/Roms.rar /content/ROM/ > /dev/null 2>&1
! python -m atari_py.import_roms /content/ROM/ > /dev/null 2>&1

#Video rendering dependences
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
# Imports libraries 
from PIL import Image
import gym
import numpy as np
import matplotlib.pyplot as plt
import argparse
import seaborn as sns
#sns.set_theme()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Permute
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import History

from rl.agents import SARSAAgent, DQNAgent
from rl.policy import BoltzmannQPolicy, MaxBoltzmannQPolicy, GreedyQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory, Memory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [ ]:
#This block does video rendering in Colab. Retrieved from https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/. 
from gym.wrappers import Monitor
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
#Defines the environment
ENV_NAME = 'Acrobot-v1'
env = gym.make(ENV_NAME)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0]
WINDOW_LENGTH = 1

In [ ]:
#Contructs model
model = Sequential()
model.add(Flatten(input_shape = (WINDOW_LENGTH,n_states)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(n_actions, activation='linear'))

#Summary of the constructed neural network
model.summary()

In [ ]:
#Builds DQN agent
memory = SequentialMemory(limit = 50_000, window_length = WINDOW_LENGTH)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, policy = policy, nb_actions = n_actions, memory=memory, 
               nb_steps_warmup = 200, target_model_update=1e-2)

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [ ]:
#Trains DQN agent
dqn_history = dqn.fit(env, nb_steps=100_000, visualize=False, verbose = 1)

In [ ]:
#Plots learning curve
smooth = 20
y = np.ones(smooth)
x = np.asarray(dqn_history.history['episode_reward'])
z = np.ones(len(x))
smoothed_rewards = np.convolve(x,y,'same') / np.convolve(z,y,'same')

plt.plot( dqn_history.history['nb_steps'] , dqn_history.history['episode_reward'],  alpha = 0.2, c='black', label = 'Raw')
plt.plot(dqn_history.history['nb_steps'] , smoothed_rewards, c = 'b' , label = 'Smoothed')
plt.title(f'DQNAgent in "{ENV_NAME}" environment')
plt.legend()
plt.xlabel('Steps');
plt.ylabel('Reward');

In [ ]:
#Saves model's weight
dqn.save_weights(f'/content/dqn_{ENV_NAME}_weights.h5f', overwrite=True)

In [ ]:
#Evaluates DQN model
dqn.test(env, nb_episodes=5, visualize=False)

In [ ]:
# Makes a video of the agent
env.close()
env = wrap_env(gym.make(ENV_NAME)) 
for i in range(1):
    score = 0
    state = env.reset()
    done = False
    while not done:
        action = dqn.forward(state)
        #action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        score+=reward
        env.render()
        dqn.backward(reward, terminal = done)
    print(f'Episode {i}, Score {score}')
env.close()
show_video()